In [1]:
from PRU import PRU_trace, TamakiOptimizer
from eunn import Unitary
import torch

In [121]:
def train_PRU():
    
    n_batch = 1
    n_qubits = 10
    n_layers = 6
    epochs = 2

    pru = PRU_trace(n_qubits, n_layers, optimizer=TamakiOptimizer(wait_time=min(2**min(n_qubits,n_layers), 1500)))
    even_layers = (n_layers+1)//2
    odd_layers = n_layers//2
    even_counts = n_qubits//2
    odd_counts = (n_qubits-1)//2
    gates = ( even_layers * even_counts + odd_layers * odd_counts ) * 2
    unitary_gen = Unitary(n_batch * gates)
    torch_optim = torch.optim.Adam(unitary_gen.parameters(), lr=0.1)
    index1 = even_layers*even_counts
    index2 = index1 + odd_layers*odd_counts
    index3 = index2 + index1

    for _ in range(epochs):
        unitaries = unitary_gen()
        unitaries = unitaries.reshape(n_batch, gates, 2, 2, 2, 2)
        unitary_dict = {}
        unitary_dict['even_layer_unitaries1'] = unitaries[:, :index1].reshape(n_batch, even_layers, even_counts, 2, 2, 2, 2)
        unitary_dict['odd_layer_unitaries1'] = unitaries[:, index1:index2].reshape(n_batch, odd_layers, odd_counts, 2, 2, 2, 2)
        unitary_dict['even_layer_unitaries2'] = unitaries[:, index2:index3].reshape(n_batch, even_layers, even_counts, 2, 2, 2, 2)
        unitary_dict['odd_layer_unitaries2'] = unitaries[:, index3:].reshape(n_batch, odd_layers, odd_counts, 2, 2, 2, 2)
        #unitary_dict = pru.layer_random_unitaries('cpu', n_batch)
        results = pru(unitary_dict)
        loss = - (results.abs()**6).mean()
        torch_optim.zero_grad(set_to_none=True)
        loss.backward()
        print(unitary_gen.angles[0])
        #print(loss)
        torch_optim.step()
        value = unitaries[0,0].reshape(4,4)
        #print(value)
        unitary1 = unitary_dict['even_layer_unitaries1'][0,-1,-1].reshape(4,4)
        unitary2 = unitary_dict['even_layer_unitaries2'][0,-1,-1].reshape(4,4)
        #print(abs(unitary1)-abs(unitary2))

In [122]:
train_PRU()

Using previously saved contraction order at  <_io.BufferedReader name='/home/minzhao.liu/Saved_Contraction_Orders/TamakiOptimizer/TraceEvaluation/wait_time_64/PRU_trace_n_10_l_6.pickle'>
tensor([[3.9192, 4.6635, 5.3250, 2.6761],
        [4.8792, 3.1118, 1.4148, 3.7182],
        [6.1518, 3.1189, 6.1233, 2.0921],
        [2.9336, 6.0892, 3.3753, 2.4000]], grad_fn=<SelectBackward0>)
tensor([[4.0192, 4.7635, 5.4250, 2.7761],
        [4.9792, 3.2118, 1.3148, 3.6182],
        [6.2518, 3.0189, 6.2233, 1.9921],
        [2.8336, 5.9892, 3.2753, 2.5000]], grad_fn=<SelectBackward0>)


In [17]:
epochs = 10

unitary_gen = Unitary(1)
torch_optim = torch.optim.Adam(unitary_gen.parameters(), lr=0.2)

for _ in range(epochs):
    a = unitary_gen()
    loss = a.abs().mean()
    torch_optim.zero_grad(set_to_none=True)
    loss.backward()
    print(loss)
    torch_optim.step()

a = a.squeeze(0)
(a@a.T.conj()).abs()

tensor(0.4755, grad_fn=<MeanBackward0>)
tensor(0.4073, grad_fn=<MeanBackward0>)
tensor(0.3484, grad_fn=<MeanBackward0>)
tensor(0.3276, grad_fn=<MeanBackward0>)
tensor(0.3186, grad_fn=<MeanBackward0>)
tensor(0.2907, grad_fn=<MeanBackward0>)
tensor(0.3007, grad_fn=<MeanBackward0>)
tensor(0.2962, grad_fn=<MeanBackward0>)
tensor(0.2678, grad_fn=<MeanBackward0>)
tensor(0.2899, grad_fn=<MeanBackward0>)


tensor([[1.0000e+00, 1.3670e-08, 8.3071e-08, 4.1650e-09],
        [1.3670e-08, 1.0000e+00, 5.6650e-09, 7.6799e-09],
        [8.3071e-08, 5.6650e-09, 1.0000e+00, 4.2742e-09],
        [4.1650e-09, 7.6799e-09, 4.2742e-09, 1.0000e+00]],
       grad_fn=<AbsBackward0>)